# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [203]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests





# Import API key
from api_keys import geoapify_key


import warnings
warnings.filterwarnings("ignore")


In [204]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", encoding="utf-8")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,76.95,69,24,12.08,PF,1681314365
1,1,isiro,2.7739,27.6160,79.38,60,100,3.15,CD,1681314365
2,2,sitka,57.0531,-135.3300,39.09,65,100,8.05,US,1681314365
3,3,yellowknife,62.4560,-114.3525,16.32,73,75,11.50,CA,1681314366
4,4,ahipara,-35.1667,173.1667,64.40,87,25,12.97,NZ,1681314366


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [205]:
#capture --no-display

# Configure the map plot
humidity = city_data_df.hvplot.points(
x="Lng",    
y="Lat",
geo=True,
title="Humidity",
frame_width = 600,  
frame_height = 360,
size=2,
scale=2,
color="Humidity",
hover_cols=["City", "Country",  "Humidity"]
)


# Display the map
humidity



:Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [206]:
#Narrow down citiesthat fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 90) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)]



# Drop any rows with null values
ideal_weather.dropna(inplace=True)
ideal_weather_df = pd.DataFrame(ideal_weather)

ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
71,71,tura,25.5198,90.2201,76.87,25,0,3.33,IN,1681314377
107,107,lexington,37.9887,-84.4777,70.99,37,0,3.44,US,1681314345
191,191,waddan,29.1614,16.1390,74.01,13,0,2.08,LY,1681314398
203,203,sulphur springs,33.1385,-95.6011,71.55,52,0,5.75,US,1681314169
232,232,tezu,27.9167,96.1667,73.98,43,0,4.94,IN,1681314407


### Step 3: Create a new DataFrame called `hotel_df`.

In [207]:
#Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()    
#hotel_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] > 70) & (ideal_weather_df["Max Temp"] < 90) & (ideal_weather_df["Wind Speed"] < 10) & (ideal_weather_df["Cloudiness"] == 0) & (ideal_weather_df["Humidity"] <=20), ["City", "Country", "Lat", "Lng", "Humidity"]].copy()

 
# Add anempty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
#hotel_df=hotel_df.assign(Hotel_Name="")     
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity
71,tura,IN,25.5198,90.2201,25
107,lexington,US,37.9887,-84.4777,37
191,waddan,LY,29.1614,16.1390,13
203,sulphur springs,US,33.1385,-95.6011,52
232,tezu,IN,27.9167,96.1667,43


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [208]:
#Set parameters to search for a hotel
radius = 10000
params = {
    "type":"lodging",
    "radius":radius,
    "filter":"circle:radius@lat,lng",
    "bias":"proximity:lat,lng",
    "key":geoapify_key,    
}
#Print a message to follow up the hotel search
print("Starting hotel search")

#Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]    
    lng = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: No hotel found
lexington - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
sulphur springs - nearest hotel: No hotel found
tezu - nearest hotel: No hotel found
nuevitas - nearest hotel: No hotel found
sabha - nearest hotel: No hotel found
tiruchchendur - nearest hotel: No hotel found
elizabeth city - nearest hotel: No hotel found
kattivakkam - nearest hotel: No hotel found
buckeye - nearest hotel: No hotel found
calama - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
eldorado - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
minab - nearest hotel: No hotel found
mizdah - nearest hotel: No hotel found
bastrop - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
numaligarh - nearest hotel: No hotel found
yarada - nearest hotel: No hotel found
imbituba - nearest hotel: No hotel found
nabinagar - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
71,tura,IN,25.5198,90.2201,25,No hotel found
107,lexington,US,37.9887,-84.4777,37,No hotel found
191,waddan,LY,29.1614,16.1390,13,No hotel found
203,sulphur springs,US,33.1385,-95.6011,52,No hotel found
232,tezu,IN,27.9167,96.1667,43,No hotel found
233,nuevitas,CU,21.5453,-77.2644,55,No hotel found
254,sabha,LY,27.0377,14.4283,9,No hotel found
255,tiruchchendur,IN,8.4833,78.1167,65,No hotel found
279,elizabeth city,US,36.2946,-76.2511,42,No hotel found
292,kattivakkam,IN,13.2167,80.3167,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [213]:

%%capture --no-display
#Configure the map plot
hotel_df = city_data_df.hvplot.points(
x="Lng",    
y="Lat",
geo=True,
title="Hotel Name",
frame_width = 500,  
frame_height =460,
size=3,
scale=3,
color="Hotel Name",
hover_cols=["City", "Country",  "Humidity"]
)


# Display the map
hotel_df 


:Points   [Lng,Lat]   (City,Country,Humidity)